# Playground with Random Forest algorithm

In [1]:
%matplotlib inline
import os,sys
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import pandas as pd
from PIL import Image

from helpers.features import *
from helpers.data_helpers import *
from helpers.image_helpers import *
from helpers.helpers import *
from helpers.constants import *
from helpers.training import train
from helpers.mask_to_submission import generate_submission_csv

done


In [2]:
TRAIN = True

In [3]:
if TRAIN : 
    trainset = AerialDataset(TRAIN_IMAGE_DATA,
                             TRAIN_LABEL_DATA,
                             range(1, TRAINING_SIZE + 1), 
                             IMG_PATCH_SIZE,
                             None)
    validationset = AerialDataset(TRAIN_IMAGE_DATA, TRAIN_LABEL_DATA, range(1 + TRAINING_SIZE, DATA_SIZE + 1), IMG_PATCH_SIZE, None, validation=True)
    testset = AerialDataset(TEST_IMAGE_DATA, TEST_LABEL_DATA, range(1, TESTING_SIZE + 1), IMG_PATCH_SIZE, None, test=True)


Loading aerial 80 images, done.
Loading 80 groundtruth images, done.
before augment
(50000, 16, 16, 3)
after augment
(50000, 16, 16, 3)

Loading aerial 20 images, done.
Loading 20 groundtruth images, done.
before augment
(12500, 16, 16, 3)
after augment
(12500, 16, 16, 3)

Loading aerial 50 images, done.
before augment
(72200, 16, 16, 3)
after augment
(72200, 16, 16, 3)


In [4]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

param_grid = {
                 'n_estimators': [100],
                 'max_leaf_nodes': [500],
                 'max_depth': [20]
             }

rfc = RandomForestClassifier(random_state=20)
#rfc_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), random_state=20, n_estimators=500)
rfc_gridsearch = GridSearchCV(rfc, param_grid=param_grid, verbose=2)

In [5]:
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=20, verbose=0, warm_start=False)

In [6]:
X = np.asarray([extract_features(trainset.train_data[i]) for i in range(len(trainset.train_data))])
X_validation = np.asarray([extract_features(validationset.train_data[i]) for i in range(len(validationset.train_data))])
X_test = np.asarray([extract_features(testset.train_data[i]) for i in range(len(testset.train_data))])
X.shape

(50000, 6)

The following cell is used in order to test tuning hyper parameters:

In [ ]:
rfc_gridsearch.fit(X, trainset.train_labels)
results = pd.DataFrame(rfc_gridsearch.cv_results_)
results.to_csv(path_or_buf='data/results_gridsearch.csv', encoding='utf8')

In [27]:
from sklearn.metrics import accuracy_score
# max_depth=None, max_leaf_nodes=1500, n_estimators=800
# max_depth=None, n_estimators=800
rfc.set_params(max_depth=30, max_leaf_nodes=1500, n_estimators=120)
rfc.fit(X, trainset.train_labels)

y_val = rfc.predict(X_validation)
accuracy = accuracy_score(validationset.train_labels, y_val, normalize=True)
accuracy, y_val

(0.78768000000000005, array([[ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.],
        ..., 
        [ 1.,  0.],
        [ 0.,  1.],
        [ 0.,  1.]]))

In [28]:
y_val, validationset.train_labels

(array([[ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.],
        ..., 
        [ 1.,  0.],
        [ 0.,  1.],
        [ 0.,  1.]]), array([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  1.]], dtype=float32))

In [30]:
y_test = rfc.predict(X_test)
y_test_final = np.zeros((y_test.shape[0], ))
y_test_final[y_test[:, 1] == 1] = 1
generate_predictions(y_test_final, concatenate=True)

In [31]:
generate_submission_csv()

./data/test/predictions/satImage_001.png
./data/test/predictions/satImage_002.png
./data/test/predictions/satImage_003.png
./data/test/predictions/satImage_004.png
./data/test/predictions/satImage_005.png
./data/test/predictions/satImage_006.png
./data/test/predictions/satImage_007.png
./data/test/predictions/satImage_008.png
./data/test/predictions/satImage_009.png
./data/test/predictions/satImage_010.png
./data/test/predictions/satImage_011.png
./data/test/predictions/satImage_012.png
./data/test/predictions/satImage_013.png
./data/test/predictions/satImage_014.png
./data/test/predictions/satImage_015.png
./data/test/predictions/satImage_016.png
./data/test/predictions/satImage_017.png
./data/test/predictions/satImage_018.png
./data/test/predictions/satImage_019.png
./data/test/predictions/satImage_020.png
./data/test/predictions/satImage_021.png
./data/test/predictions/satImage_022.png
./data/test/predictions/satImage_023.png
./data/test/predictions/satImage_024.png
./data/test/pred